In [1]:
import sys
import numpy
from matplotlib import pyplot
from tensorflow import keras
from keras.utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.losses import BinaryCrossentropy
import time
# from keras.metrics import Accuracy
from tensorflow.keras.metrics import BinaryAccuracy
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import io

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

def define_model():
    _input = Input((224,224,3)) 

    conv1  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(_input)
    conv2  = Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu")(conv1)
    pool1  = MaxPooling2D((2, 2))(conv2)

    conv3  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(pool1)
    conv4  = Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu")(conv3)
    pool2  = MaxPooling2D((2, 2))(conv4)

    conv5  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(pool2)
    conv6  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv5)
    conv7  = Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu")(conv6)
    pool3  = MaxPooling2D((2, 2))(conv7)

    conv8  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool3)
    conv9  = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv8)
    conv10 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv9)
    pool4  = MaxPooling2D((2, 2))(conv10)

    conv11 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(pool4)
    conv12 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv11)
    conv13 = Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu")(conv12)
    pool5  = MaxPooling2D((2, 2))(conv13)

    flat   = Flatten()(pool5)
    dense1 = Dense(4096, activation="relu")(flat)
    dense2 = Dense(4096, activation="relu")(dense1)
    output = Dense(1, activation="sigmoid")(dense2)

    vgg16_model  = Model(inputs=_input, outputs=output)
    opt = SGD(learning_rate=0.001, momentum=0.9)
    vgg16_model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return vgg16_model

# create data generator
datagen = ImageDataGenerator(featurewise_center=True)
# specify imagenet mean values for centering
datagen.mean = [123.68, 116.779, 103.939]
# prepare iterator
train_it = datagen.flow_from_directory('images/train/',
class_mode='binary', batch_size=64, target_size=(224, 224))
test_it = datagen.flow_from_directory('images/test/',
class_mode='binary', batch_size=64, target_size=(224, 224))

model = define_model()
num_epochs = 10
binary_crossentropy = BinaryCrossentropy()
optimizer = SGD(learning_rate=0.001, momentum=0.9)
# accuracy_metric = Accuracy()
accuracy_metric = BinaryAccuracy()

batch_size=64
num_train_images=160 # for both classes
num_test_images=40 # for both classes

train_loop_iter=numpy.ceil(num_train_images/batch_size)
test_loop_iter=numpy.ceil(num_test_images/batch_size)
start=time.time()
for epoch in range(num_epochs):
    # Iterate through training set
    for batch_idx, (x, y) in enumerate(train_it):
        # print(f"Batch {batch_idx}: x shape = {x.shape}, y shape = {y.shape}")
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss = binary_crossentropy(y, y_pred)

        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        accuracy_metric.update_state(y, y_pred)
        train_loop_iter=train_loop_iter-1
        if train_loop_iter==0:
            train_loop_iter=numpy.ceil(num_train_images/batch_size)
            break

    print(f"Train Loss (Epoch {epoch + 1}): {loss.numpy():.4f}")
    print(f"Train Accuracy (Epoch {epoch + 1}): {accuracy_metric.result().numpy()*100:.4f}%")

    # Reset accuracy in between epochs (and for testing and test)
    accuracy_metric.reset_state()

end=time.time()

# Iterate through test set
for batch_idx, (x, y) in enumerate(test_it):
    y_pred = model(x, training=False)
    loss = binary_crossentropy(y, y_pred)
    accuracy_metric.update_state(y, y_pred)
    test_loop_iter=test_loop_iter-1
    if test_loop_iter==0:
        test_loop_iter=numpy.ceil(num_test_images/batch_size)
        break
print(f"Test Loss: {loss.numpy():.4f}")
print(f"Test Accuracy: {accuracy_metric.result().numpy()*100:.4f}%")

print(f"Training Time: {end-start}s")
model.summary()

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


C:\Users\asus\AppData\Roaming\Python\Python311\site-packages\PIL\Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Train Loss (Epoch 1): 0.6715
Train Accuracy (Epoch 1): 56.8750%
Train Loss (Epoch 2): 0.6759
Train Accuracy (Epoch 2): 50.0000%
Train Loss (Epoch 3): 0.6677
Train Accuracy (Epoch 3): 58.1250%
Train Loss (Epoch 4): 0.6572
Train Accuracy (Epoch 4): 65.0000%
Train Loss (Epoch 5): 0.6149
Train Accuracy (Epoch 5): 71.8750%
Train Loss (Epoch 6): 0.5550
Train Accuracy (Epoch 6): 73.7500%
Train Loss (Epoch 7): 0.5697
Train Accuracy (Epoch 7): 74.3750%
Train Loss (Epoch 8): 0.6284
Train Accuracy (Epoch 8): 74.3750%
Train Loss (Epoch 9): 0.5254
Train Accuracy (Epoch 9): 76.2500%
Train Loss (Epoch 10): 0.4699
Train Accuracy (Epoch 10): 77.5000%
Test Loss: 0.5277
Test Accuracy: 72.5000%
Training Time: 773.6306853294373s


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         4,097 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,264,641 (512.18 MB)

 Trainable params: 134,264,641 (512.18 MB)

 Non-trainable params: 0 (0.00 B)